In [31]:
import sys
sys.path.insert(0, '../')

In [32]:
from collections import Counter

import wlkernel
import rdflib
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [33]:
rdf_graph = rdflib.Graph().parse('../tests/resources/aifbfixed_complete.n3', format='n3')

In [34]:
Counter(str(o) for s, p, o in rdf_graph if 'affiliation' in str(p))

Counter({'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id4instance': 16,
         'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance': 73,
         'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance': 60,
         'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance': 28,
         'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id5instance': 1})

In [35]:
instances_class_map = {
    str(s): str(o) for s, p, o in rdf_graph 
    if 'affiliation' in str(p)
    and 'id5instance' not in str(o)
}
len(instances_class_map)

177

In [36]:
instances = list(instances_class_map)
len(instances)

177

In [37]:
classes = set(instances_class_map.values())
len(classes)

4

In [38]:
triples = list(
    (str(s), str(p), str(o)) for s, p, o in rdf_graph
    if 'affiliation' not in str(p)
    and 'employs' not in str(p)
    and 'member' not in str(p)
    and 'head' not in str(p)
)
len(triples)

28699

In [39]:
wlrdf_graph = wlkernel.WLRDFGraph(triples, instances, 2)
len(wlrdf_graph.instance_nodes)

177

In [40]:
kernel_matrix = wlkernel.wlrdf_kernel_matrix(wlrdf_graph, instances, 1)
len(kernel_matrix)

177

In [18]:
X = list(instances_class_map)
y = list(instances_class_map.values())
for i in range(len(X)):
    assert y[i] == instances_class_map[X[i]]
len(y)

177

In [19]:
classifier = svm.SVC(kernel='precomputed')
classifier.fit(kernel_matrix, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='precomputed', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [20]:
pr = classifier.predict(kernel_matrix)
print('Accuracy on training set')
accuracy_score(y, pr)

Accuracy on training set


0.6214689265536724